In [1]:
def innerL2(gfu,mesh):
    return Integrate(InnerProduct(gfu,gfu),mesh)
def L2error(gfu,exact,mesh):
    return sqrt(innerL2(gfu-exact,mesh))

In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import pickle
from netgen.occ import *
import netgen.meshing as ngm
import numpy as np
from ngsolve.webgui import *
from netgen.geom2d import unit_square
import params
geom = SplineGeometry()
w = 5
l = 1  
geo = SplineGeometry()

pnts =[(0,0),
       (w,0),
       (w,l),
       (0,l)]

p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]

curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]

[geo.Append(c,bc=bc) for c,bc in curves]
n = 7
mesh = Mesh(geo.GenerateMesh(maxh=2**(-n)))

phi0 = 0.5
chi = 0.2
N = params.N
KBTV = params.KBTV
gamma = 0.0011455

def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = log(1-phi) + phi + chi*phi**2
    edp = gamma * InnerProduct(F,dv) + InnerProduct(H_prime * J * invF,dv)
    return edp  # mean zero condition

def Solver_freeswell(BF, gfu, tol=1e-8, maxiter=250, damp = 0.5):
    """
    Solves the problem
    """
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    history = GridFunction(fes, multidim = 0)
    # here we may need to add another loop
   
    for iter in range(maxiter):
        # Prints before the iteration: number of it, residual, energy
        print("Energy: ", BF.Energy(gfu.vec), "Residual: ", sqrt(abs(InnerProduct(res,res))), "Iteration: ", iter)
        BF.Apply(gfu.vec, res)
        BF.AssembleLinearization(gfu.vec)
        inv = BF.mat.Inverse(freedofs = fes.FreeDofs())        
        w.data = damp * inv * res
        gfu.vec.data -= w
        history.AddMultiDimComponent(gfu.vec)
        if sqrt(abs(InnerProduct(w,res))) < tol:
            print("Converged")
            break
    return gfu, history


fes= VectorH1(mesh, order=2,dirichletx = "left", dirichlety = "bottom")
u,v = fes.TnT()
F = Id(2) + Grad(u)
BF = BilinearForm(fes, symmetric=False)
BF += Gel_energy_EDP(F).Compile() * dx
BF.Assemble()
print("Number of dofs:", fes.ndof)
gfu = GridFunction(fes)
gfu, history = Solver_freeswell(BF, gfu, tol=1e-6, maxiter=20, damp = 0.5)
pickle.dump(gfu, open(f"gfu_n_{n}.p", "wb"))

Number of dofs: 760970
Energy:  0.0 Residual:  0.0 Iteration:  0
Energy:  -0.04869646049729223 Residual:  0.04360144694877884 Iteration:  1
Energy:  -0.06810157970928528 Residual:  0.027713801685856475 Iteration:  2
Energy:  -0.07079857513584503 Residual:  0.0175402033950956 Iteration:  3
Energy:  -0.06464455742941844 Residual:  0.011049662017835022 Iteration:  4
Energy:  -0.054343732039441005 Residual:  0.006917349711077955 Iteration:  5
Energy:  -0.04264345248712801 Residual:  0.004289411994967885 Iteration:  6
Energy:  -0.03120013746818388 Residual:  0.0026204587762081633 Iteration:  7
Energy:  -0.021131694930458966 Residual:  0.0015651434518419007 Iteration:  8
Energy:  -0.013192485887079164 Residual:  0.000906224972040265 Iteration:  9
Energy:  -0.007639136923084111 Residual:  0.0005057281976249725 Iteration:  10
Energy:  -0.004171649673731358 Residual:  0.00027220984912232896 Iteration:  11
Energy:  -0.0021909656069839436 Residual:  0.00014232325707115614 Iteration:  12
Energy:  

In [3]:
ns = [1,2,3,4,5,6]
u = pickle.load(open(f"gfu_n_{7}.p", "rb"))
errors = []
for n in ns:
    g = pickle.load(open(f"gfu_n_{n}.p", "rb"))
    errors.append(L2error(g,u,mesh))

In [5]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene